# Pairwise MLP with PU

Train a pairwise MLP model in a pile-up environment.

## Problem setup

The reconstruction process consists of the following steps:

### 1. Select tracksters above a certain energy threshold

Select high energy trackster we want to run the smoothing around.
The energy threshold is a fine-tined parameter, generally, 10 - 50 GeV seems to work well.

**Result**: Selected Tracksters


### 2. Get tracksters in their cylindrical neighborhood
- The cylinder is defined along the axis connecting the trackster barycenter to 0,0,0 and a selected radius (e.g. 10cm)
- Result: Trackster Candidates, Candidate Pairs

### 3. For each Candidate Pair, decide whether the two tracksters should be connected
- For each Trackster Candidate, chose the Selected Trackster with the highest predicted score (likelihood)


## Data location

Set `ds_name` to the dataset name, and point `raw_dir` to the directory containing ntuplized `.root` files.
Processed `PyTorch` datasets will be placed in the `data_root` folder. The program will verify if the requested datasets exist already. Make sure the directories exist.

In [ ]:
ds_name = "CloseByPion200PU"
raw_dir = "/home/ecuba/data/CloseByPion200PU"
data_root = "/home/ecuba/data/processed"
model_dir = "/home/ecuba/data/models"

## Initialization

In [ ]:
import torch
import torch.nn as nn
import torch.onnx

import numpy as np
import onnxruntime

from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import random_split, DataLoader

from matplotlib import rc
import matplotlib.pyplot as plt
rc('font',**{'family':'sans-serif','sans-serif':['DejaVu Sans'],'size': 12})
rc('mathtext',**{'default':'regular'})
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from reco.loss import QualityFocalLoss
from reco.dataset_pair import TracksterPairs
from reco.training import train_mlp, roc_auc, precision_recall_curve

In [ ]:
# CUDA Setup
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Load or process the dataset

At least `N_FILES=100` root files is recommended to have a large enough training sample.
The following parameters need to be set:
- `bigT_e_th`: energy threshold to select tracksters for smoothing
- `radius`: radius of the cylinder around the selected tracksters
- `pileup`: needs to be set to `true` for pile-up datasets, the selection of the tracksters is limited to ones that overlap with a simtrackster

In [ ]:
ds = TracksterPairs(
    ds_name,
    data_root,
    raw_dir,
    N_FILES=20,
    radius=15,
    pileup=True,
    bigT_e_th=10,
)

ds[0][0]
ds.x.shape

## Split the dataset into train and validation

In [ ]:
decision_threshold = 0.7

print(ds.x.shape)
print("Positive:", int((ds.y >= decision_threshold).type(torch.int).sum()))
print("Negative:", int((ds.y < decision_threshold).type(torch.int).sum()))

balance =  float(sum(ds.y > decision_threshold) / len(ds.y))
print(f"dataset balance: {balance*100:.2f}% / {(1-balance)*100:.2f}%") 

In [ ]:
# Fraction of the pairs to be left for validation
val_set_fraction = 0.1

ds_size = len(ds)
val_set_size = ds_size // int(1. / val_set_fraction)
train_set_size = ds_size - val_set_size
train_set, val_set = random_split(ds, [train_set_size, val_set_size])
print(f"Train samples: {len(train_set)}, Validation samples: {len(val_set)}")

train_dl = DataLoader(train_set, batch_size=32, shuffle=True)
val_dl = DataLoader(val_set, batch_size=32, shuffle=True)

## Model configuration

Configure model and training setup.

In [ ]:
epochs = 101

hdim1 = 256
hdim2 = 128

dropout_pr = 0.2

In [ ]:
model = nn.Sequential(
    nn.LayerNorm(ds.x.shape[1]),      # normalization as a part of the network
    nn.Linear(ds.x.shape[1], hdim1),
    nn.Sigmoid(),
    nn.Linear(hdim1, hdim2),
    nn.Sigmoid(),
    nn.Linear(hdim2, 1),
    nn.Dropout(p=dropout_pr),
)
model = model.to(device)
model_path = f"{model_dir}/PairWiseMLP.{hdim1}.{hdim2}.{epochs}e-{ds_name}.r{ds.RADIUS}.e{ds.bigT_e_th}.f{ds.N_FILES}.pt"

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
scheduler = CosineAnnealingLR(optimizer, epochs, eta_min=1e-5)
loss_obj = QualityFocalLoss(gamma=2)

## Training

In [ ]:
#%%script echo skipping

for epoch in range(epochs):
    loss = train_mlp(model, device, optimizer, train_dl, loss_obj)
    train_auc = roc_auc(model, device, train_dl)
    val_auc = roc_auc(model, device, val_dl)
    scheduler.step()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch}, loss: {loss:.4f}, train auc: {train_auc:.4f}, val auc: {val_auc:.4f}')

torch.save(model.state_dict(), model_path)

In [ ]:
model.load_state_dict(torch.load(
    model_path,
    map_location=device
))
model = model.to(device)

In [ ]:
roc_auc(model, device, val_dl)

In [ ]:
precision_recall_curve(model, device, val_dl, step=3)

# ONNX export

Export the trained model into ONNX and verify the output.

In [ ]:
onnx_filepath = f"{model_dir}/PairWiseMLP.{hdim1}.{hdim2}.{epochs}e-{ds_name}.{ds.RADIUS}.{ds.SCORE_THRESHOLD}.{ds.N_FILES}f.onnx"

torch.onnx.export(
    model,                      # model to be exported
    ds[0][0].reshape(1, -1),    # example input (add batch dimension)
    onnx_filepath,
    export_params=True,
    opset_version=10,
    do_constant_folding=True,
    input_names=['features'],      # the model's input names
    output_names=['output'],    # the model's output names
    dynamic_axes={              # variable length axes
        'features' : {0 : 'batch_size'},    
        'output' : {0 : 'batch_size'}
    }
)

In [ ]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

ort_session = onnxruntime.InferenceSession(onnx_filepath)
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(ds[:16][0])}
ort_outs = ort_session.run(None, ort_inputs)
torch_out = model(ds[:16][0])
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)
print("Exported model has been tested with ONNXRuntime, and the result looks good!")

## Evaluation

In [ ]:
from reco.data import get_event_data
from reco.evaluation import model_evaluation
from reco.dummy import DummyPleaser

file_name = f"{raw_dir}/test/test_samples_1.root"
cluster_data, trackster_data, simtrackster_data, assoc_data = get_event_data(file_name, pileup=True)

In [ ]:
r_ranges = [3, 5, 10, 15]
results = []

max_events = 10
clue3D_F = []
target_F = []
naive_reco_F = []
model_reco_F = []
for r in r_ranges:
    print(f" --- Radius threshold: {r} ---")
    result = model_evaluation(
        cluster_data,
        trackster_data,
        simtrackster_data,
        assoc_data,
        DummyPleaser(),
        decision_th=decision_threshold,
        radius=r,
        max_events=max_events,
        bigT_e_th=10,
        pileup=True
    )
    clue3D_F.append(np.sum(np.array(result["clue3d_to_sim"])[:,2]) / max_events)
    target_F.append(np.sum(np.array(result["target_to_sim"])[:,2]) / max_events)
    naive_reco_F.append(np.sum(np.array(result["reco_to_sim"])[:,2]) / max_events)

    result = model_evaluation(
        cluster_data,
        trackster_data,
        simtrackster_data,
        assoc_data,
        model.to("cpu"),
        decision_th=decision_threshold,
        radius=r,
        max_events=max_events,
        bigT_e_th=10,
        pileup=True
    )
    model_reco_F.append(np.sum(np.array(result["reco_to_sim"])[:,2]) / max_events)

In [ ]:
fig = plt.figure(figsize=(8, 5))
ax = fig.add_subplot(111)

ax.plot(r_ranges[:6], (np.array(target_F) - np.array(clue3D_F))[:6], '--s', label="target", c="#D55E00")
ax.plot(r_ranges[:6], (np.array(model_reco_F) - np.array(clue3D_F))[:6], '-o', label="mlp", c="#56B4E9")
ax.plot(r_ranges[:6], (np.array(naive_reco_F) - np.array(clue3D_F))[:6], '-v', label="naive", c="#E69F00")
ax.axhline(max(np.array(naive_reco_F) - np.array(clue3D_F)), label="baseline", c="lightgray", linestyle="--")

ax.legend()
ax.set_xlabel("Neighborhood radius (cm)")
ax.set_ylabel("$\Delta F_{0.5}$")
plt.show()